# Vehicle Detection Using Meta-Classifiers (Deep Learning)
---
Deep learning is a subset of machine learning that focuses on training artificial neural networks to learn and make intelligent decisions or predictions from data. It involves training neural networks with multiple layers to automatically learn hierarchical representations of the data. There are several algorithms in deep learning that can be used for classification problem.

- Importing Libraries

In [ ]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import tensorflow as tf
Total_time = time.time()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

## Data Preprocessing

In [ ]:
vehicle_folder = "/kaggle/input/vehicle-detection-image-set/data/vehicles"
nonvehicle_folder = "/kaggle/input/vehicle-detection-image-set/data/non-vehicles"
data = []
labels = []

# Process vehicle images
for filename in os.listdir(vehicle_folder):
    if filename.endswith('.png'):
        image_path = os.path.join(vehicle_folder, filename)
        image = cv2.imread(image_path)  #Reading the image
        data.append(image)
        labels.append('vehicle')

# Process non-vehicle images
for filename in os.listdir(nonvehicle_folder):
    if filename.endswith('.png'):
        image_path = os.path.join(nonvehicle_folder, filename)
        image = cv2.imread(image_path)  # Read the image
        data.append(image)
        labels.append('non-vehicle')

data = np.array(data)
labels = np.array(labels)

In [ ]:
#Using scikit-learn to split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=40)
print(len(X_train),len(X_test))

14208 3552


In [ ]:
label_mapping = {'vehicle': 1, 'non-vehicle': 0}

# Convert labels to numerical values
y_train_numeric = [label_mapping[label] for label in y_train]
y_test_numeric = [label_mapping[label] for label in y_test]

y_train_numeric = np.array(y_train_numeric)
y_test_numeric = np.array(y_test_numeric)

In [ ]:
print(len(X_train))
print(len(y_train_numeric))

14208
14208


---
## 1. CNN
Convolutional Neural Network (CNN) is the extended version of artificial neural networks (ANN) which is predominantly used to extract the feature from the grid-like matrix dataset. For example visual datasets like images or videos where data patterns play an extensive role.
- Convolution layers consist of a set of learnable filters (or kernels) having small widths and heights and the same depth as that of input volume (3 if the input layer is image input).
- For example, if we have to run convolution on an image with dimensions 34x34x3. The possible size of filters can be axax3, where ‘a’ can be anything like 3, 5, or 7 but smaller as compared to the image dimension.
- During the forward pass, we slide each filter across the whole input volume step by step where each step is called stride (which can have a value of 2, 3, or even 4 for high-dimensional images) and compute the dot product between the kernel weights and patch from input volume.
- As we slide our filters we’ll get a 2-D output for each filter and we’ll stack them together as a result, we’ll get output volume having a depth equal to the number of filters. The network will learn all the filters.


In [ ]:
from tensorflow import keras
model_cnn = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model_cnn.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
import time
Time_cnn = time.time()
early_stopping = EarlyStopping(monitor='accuracy', patience=8, restore_best_weights=True)

model_cnn.fit(X_train, y_train_numeric, epochs=100, callbacks=[early_stopping])
Time_cnn = time.time() - Time_cnn

Epoch 1/100
444/444 [==============================] - 17s 6ms/step - loss: 0.7085 - accuracy: 0.9270
Epoch 2/100
444/444 [==============================] - 2s 4ms/step - loss: 0.0611 - accuracy: 0.9792
Epoch 3/100
444/444 [==============================] - 2s 5ms/step - loss: 0.0367 - accuracy: 0.9878
Epoch 4/100
444/444 [==============================] - 2s 4ms/step - loss: 0.0324 - accuracy: 0.9893
Epoch 5/100
444/444 [==============================] - 2s 4ms/step - loss: 0.0418 - accuracy: 0.9851
Epoch 6/100
444/444 [==============================] - 2s 4ms/step - loss: 0.0201 - accuracy: 0.9930
Epoch 7/100
444/444 [==============================] - 2s 4ms/step - loss: 0.0386 - accuracy: 0.9873
Epoch 8/100
444/444 [==============================] - 2s 5ms/step - loss: 0.0179 - accuracy: 0.9948
Epoch 9/100
444/444 [==============================] - 2s 4ms/step - loss: 0.0233 - accuracy: 0.9935
Epoch 10/100
444/444 [==============================] - 2s 4ms/step - loss: 0.0366 - accur

In [ ]:
loss, accuracy = model_cnn.evaluate(X_test, y_test_numeric)

111/111 [==============================] - 0s 3ms/step - loss: 0.7644 - accuracy: 0.9412


---
## 2. LSTM
 LSTM stands for long short-term memory networks, used in the field of Deep Learning. It is a variety of recurrent neural networks (RNNs) that are capable of learning long-term dependencies, especially in sequence prediction problems. LSTM has feedback connections, i.e., it is capable of processing the entire sequence of data, apart from single data points such as images. This finds application in speech recognition, machine translation, etc. LSTM is a special kind of RNN, which shows outstanding performance on a large variety of problems.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


image_height, image_width, num_channels = 64, 64, 3
num_timesteps = image_height * image_width * num_channels  # Flatten the image as a sequence


# Build the LSTM model for image feature analysis
model_lstm = keras.Sequential([
    keras.layers.LSTM(64, input_shape=(num_timesteps, 1)),
    keras.layers.Dense(2, activation='softmax')
])

model_lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape the input data to flatten the images as sequences
X_train_lstm = X_train.reshape(-1, num_timesteps, 1)
X_test_lstm = X_test.reshape(-1, num_timesteps, 1)



In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
Time = 0
Time_lstm = time.time()
early_stopping = EarlyStopping(monitor='accuracy', patience=8, restore_best_weights=True)

model_lstm.fit(X_train_lstm, y_train_numeric,batch_size=250, epochs=100, callbacks=[early_stopping])
Time_lstm = time.time() - Time_lstm


Epoch 1/100
57/57 [==============================] - 33s 527ms/step - loss: 0.6141 - accuracy: 0.7119
Epoch 2/100
57/57 [==============================] - 30s 534ms/step - loss: 0.5028 - accuracy: 0.8119
Epoch 3/100
57/57 [==============================] - 31s 542ms/step - loss: 0.3624 - accuracy: 0.8596
Epoch 4/100
57/57 [==============================] - 31s 547ms/step - loss: 0.6422 - accuracy: 0.6942
Epoch 5/100
57/57 [==============================] - 32s 554ms/step - loss: 0.3566 - accuracy: 0.8630
Epoch 6/100
57/57 [==============================] - 32s 555ms/step - loss: 0.3151 - accuracy: 0.8770
Epoch 7/100
57/57 [==============================] - 31s 549ms/step - loss: 0.3017 - accuracy: 0.8806
Epoch 8/100
57/57 [==============================] - 31s 552ms/step - loss: 0.2920 - accuracy: 0.8850
Epoch 9/100
57/57 [==============================] - 31s 550ms/step - loss: 0.2849 - accuracy: 0.8882
Epoch 10/100
57/57 [==============================] - 31s 551ms/step - loss: 0.262

In [ ]:
loss, accuracy = model_lstm.evaluate(X_test_lstm, y_test_numeric)

111/111 [==============================] - 15s 129ms/step - loss: 0.1646 - accuracy: 0.9400


---
# 3. GRU
- Gated Recurrent Unit (GRU) is a type of recurrent neural network (RNN) that was introduced as a simpler alternative to Long Short-Term Memory (LSTM) networks. Like LSTM, GRU can process sequential data such as text, speech, and time-series data.
- The basic idea behind GRU is to use gating mechanisms to selectively update the hidden state of the network at each time step. The gating mechanisms are used to control the flow of information in and out of the network. The GRU has two gating mechanisms, called the reset gate and the update gate.
- The reset gate determines how much of the previous hidden state should be forgotten, while the update gate determines how much of the new input should be used to update the hidden state. The output of the GRU is calculated based on the updated hidden state.

In [ ]:
model_gru = keras.Sequential([
    keras.layers.GRU(64, input_shape=(num_timesteps, 1)),
    keras.layers.Dense(2, activation='softmax')
])

model_gru.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape the input data to flatten the images as sequences
X_train_gru = X_train.reshape(-1, num_timesteps, 1)
X_test_gru = X_test.reshape(-1, num_timesteps, 1)

In [ ]:
Time_gru = time.time()
model_gru.fit(X_train_lstm, y_train_numeric,batch_size=250, epochs=100, callbacks=[early_stopping])
Time_gru = time.time() - Time_gru


Epoch 1/100
57/57 [==============================] - 31s 496ms/step - loss: 0.5441 - accuracy: 0.7218
Epoch 2/100
57/57 [==============================] - 28s 498ms/step - loss: 0.3211 - accuracy: 0.8668
Epoch 3/100
57/57 [==============================] - 28s 493ms/step - loss: 0.2254 - accuracy: 0.9161
Epoch 4/100
57/57 [==============================] - 28s 500ms/step - loss: 0.1874 - accuracy: 0.9336
Epoch 5/100
57/57 [==============================] - 28s 499ms/step - loss: 0.3285 - accuracy: 0.8642
Epoch 6/100
57/57 [==============================] - 28s 498ms/step - loss: 0.2137 - accuracy: 0.9200
Epoch 7/100
57/57 [==============================] - 29s 503ms/step - loss: 0.1777 - accuracy: 0.9348
Epoch 8/100
57/57 [==============================] - 28s 498ms/step - loss: 0.1604 - accuracy: 0.9412
Epoch 9/100
57/57 [==============================] - 28s 497ms/step - loss: 0.1464 - accuracy: 0.9474
Epoch 10/100
57/57 [==============================] - 28s 493ms/step - loss: 0.145

In [ ]:
loss, accuracy = model_gru.evaluate(X_test_gru, y_test_numeric)

111/111 [==============================] - 15s 128ms/step - loss: 0.0709 - accuracy: 0.9749


---
# 4. Multilayer Preceptron
    
 The multilayer perceptron (MLP) neural network is the most popular feedforward neural network widely used to tackle different classification and prediction problems. The successful behavior of MLP depends on the proper configurations of its input parameters (i.e., weights and biases), which are adjusted in the learning process using a gradient-based mechanism.

In [ ]:
model_mlp = Sequential([
    keras.layers.Flatten(input_shape=(image_height, image_width, num_channels)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model_mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='accuracy', patience=8, restore_best_weights=True)

Time_mlp = time.time()
model_mlp.fit(X_train, y_train_numeric,batch_size=250, epochs=100, callbacks=[early_stopping])
Time_mlp = time.time() - Time_mlp


Epoch 1/100
57/57 [==============================] - 2s 5ms/step - loss: 8.0114 - accuracy: 0.7323
Epoch 2/100
57/57 [==============================] - 0s 5ms/step - loss: 1.0802 - accuracy: 0.8390
Epoch 3/100
57/57 [==============================] - 0s 5ms/step - loss: 1.2694 - accuracy: 0.8266
Epoch 4/100
57/57 [==============================] - 0s 5ms/step - loss: 1.2330 - accuracy: 0.8326
Epoch 5/100
57/57 [==============================] - 0s 5ms/step - loss: 1.5696 - accuracy: 0.8038
Epoch 6/100
57/57 [==============================] - 0s 5ms/step - loss: 0.4723 - accuracy: 0.8967
Epoch 7/100
57/57 [==============================] - 0s 5ms/step - loss: 0.2911 - accuracy: 0.9082
Epoch 8/100
57/57 [==============================] - 0s 5ms/step - loss: 0.2515 - accuracy: 0.9210
Epoch 9/100
57/57 [==============================] - 0s 5ms/step - loss: 0.2629 - accuracy: 0.9130
Epoch 10/100
57/57 [==============================] - 0s 5ms/step - loss: 0.2649 - accuracy: 0.9127
Epoch 11/

In [ ]:
loss, accuracy = model_mlp.evaluate(X_test, y_test_numeric)

111/111 [==============================] - 0s 2ms/step - loss: 0.1953 - accuracy: 0.9336


---
# 5. TCN
 It is inspired by convolutional architectures for sequential data and combines simplicity, autoregressive prediction, and very long memory. The TCN is designed from two basic principles:

- The convolutions are causal, meaning that there is no information leakage from future to past. The architecture can take a sequence of any length and map it to an output sequence of the same length just as with an RNN.
- To achieve the first point, the TCN uses causal convolutions, i.e., convolutions where an output at time t is convolved only with elements from time t and earlier in the previous layer. To accomplish the second point, the TCN uses a 1D fully-convolutional network architecture, where each hidden layer is the same length as the input layer.

In [ ]:
!pip install -q keras-tcn --no-dependencies


In [ ]:
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN


input_shape = (64, 192)  # Sequence length (time steps) x Number of features

# Build the TCN model
model_tcn = Sequential([
    TCN(64, kernel_size=3, padding='causal', activation='relu', input_shape=input_shape),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_tcn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


X_train_tcn = X_train.reshape(-1, input_shape[0], input_shape[1])


# Train the model
Time_tcn= time.time()
model_tcn.fit(X_train_tcn, y_train_numeric, batch_size=250, epochs=100, callbacks=[early_stopping])
Time_tcn = time.time() - Time_tcn


Epoch 1/100
57/57 [==============================] - 11s 51ms/step - loss: 27.1203 - accuracy: 0.7969
Epoch 2/100
57/57 [==============================] - 1s 22ms/step - loss: 2.0276 - accuracy: 0.8822
Epoch 3/100
57/57 [==============================] - 1s 22ms/step - loss: 1.1941 - accuracy: 0.9086
Epoch 4/100
57/57 [==============================] - 1s 22ms/step - loss: 0.7717 - accuracy: 0.9276
Epoch 5/100
57/57 [==============================] - 1s 22ms/step - loss: 0.5185 - accuracy: 0.9431
Epoch 6/100
57/57 [==============================] - 1s 22ms/step - loss: 0.4905 - accuracy: 0.9438
Epoch 7/100
57/57 [==============================] - 1s 23ms/step - loss: 0.3298 - accuracy: 0.9593
Epoch 8/100
57/57 [==============================] - 1s 22ms/step - loss: 0.2425 - accuracy: 0.9655
Epoch 9/100
57/57 [==============================] - 1s 22ms/step - loss: 0.2632 - accuracy: 0.9620
Epoch 10/100
57/57 [==============================] - 1s 22ms/step - loss: 0.1865 - accuracy: 0.97

In [ ]:
len(X_train_tcn)

14208

In [ ]:
X_test_tcn = X_test.reshape(-1,input_shape[0],input_shape[1])
loss, accuracy = model_tcn.evaluate(X_test_tcn, y_test_numeric)

111/111 [==============================] - 1s 4ms/step - loss: 0.4325 - accuracy: 0.9620


---
# Perfomance Metrics
- Accuracy
- Precision
- Recall
- F1 Score
- AUC
- Execution Time

In [ ]:
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def evaluate_trained_model(model, X_test, y_test):
    # Before evaluating the model, record the start time
    start_time = time.time()

    # Predict on the test data
    y_pred = model.predict(X_test)

    # Convert predicted probabilities to class labels (0 or 1) if the model's final activation function is 'sigmoid'
    y_pred_classes = (y_pred > 0.5).astype(int)

    # Calculate the metrics
    accuracy = accuracy_score(y_test, y_pred_classes)
    precision = precision_score(y_test, y_pred_classes)
    recall = recall_score(y_test, y_pred_classes)
    f1 = f1_score(y_test, y_pred_classes)
    auc = roc_auc_score(y_test, y_pred)
    execution_time = time.time() - start_time

    # Create a dictionary to store the results
    results = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "AUC": auc,
    }

    print(f"Accuracy: {100*results['Accuracy']}")
    print(f"Precision: {100*results['Precision']}")
    print(f"Recall: {100*results['Recall']}")
    print(f"F1 Score: {100*results['F1 Score']}")
    print(f"AUC: {100*results['AUC']}")

In [ ]:
import time
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score

def evaluate_lstm_gru_model(model, x_data, y_true):
    y_pred_probs = model.predict(x_data)

    y_pred_positive_probs = y_pred_probs[:, 1]

    y_pred_binary = np.where(y_pred_positive_probs > 0.5, 1, 0)

    # True Positives, False Positives, True Negatives, False Negatives
    tp = np.sum((y_true == 1) & (y_pred_binary == 1))
    fp = np.sum((y_true == 0) & (y_pred_binary == 1))
    tn = np.sum((y_true == 0) & (y_pred_binary == 0))
    fn = np.sum((y_true == 1) & (y_pred_binary == 0))

    # Accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn)

    # Precision
    precision = tp / (tp + fp)

    # Recall
    recall = tp / (tp + fn)

    # F1 Score
    f1 = 2 * (precision * recall) / (precision + recall)
    auc = roc_auc_score(y_true, y_pred_binary)


    # Print the calculated metrics
    print(f"Accuracy: {accuracy * 100}%")
    print(f"Precision: {precision * 100}%")
    print(f"Recall: {recall * 100}%")
    print(f"F1 Score: {f1 * 100}%")
    print(f"AUC: {auc * 100}%")



### 1. CNN


In [ ]:
evaluate_trained_model(model_cnn,X_test,y_test_numeric)
print(f"Evaluated Time: {Time_cnn} seconds")

111/111 [==============================] - 0s 2ms/step
Accuracy: 94.115990990991
Precision: 96.45006016847172
Recall: 91.44324015972617
F1 Score: 93.87994143484626
AUC: 98.09411135742205
Evaluated Time: 57.80364775657654 seconds


### 2. LSTM

In [ ]:
np.count_nonzero(y_test_numeric == 0)

1799

In [ ]:
np.count_nonzero(y_test_numeric == 1)

1753

In [ ]:
y_test_numeric[2500]

1

In [ ]:
y_test_numeric.shape

(3552,)

In [ ]:
y_test_numeric.dtype

dtype('int64')

In [ ]:
evaluate_lstm_gru_model(model_lstm,X_test_lstm,y_test_numeric)
print(f"Evaluated Time: {Time_lstm/60} minutes")

111/111 [==============================] - 15s 134ms/step
Accuracy: 94.00337837837837%
Precision: 96.27403846153845%
Recall: 91.38619509412436%
F1 Score: 93.76646180860403%
AUC: 93.96991800287095%
Evaluated Time: 53.39798504114151 minutes


### 3. GRU

In [ ]:
evaluate_lstm_gru_model(model_gru,X_test_gru,y_test_numeric)
print(f"Evaluated Time: {Time_gru/60} minutes")

111/111 [==============================] - 14s 127ms/step
Accuracy: 97.49436936936937%
Precision: 96.37681159420289%
Recall: 98.63091842555619%
F1 Score: 97.49083732731886%
AUC: 97.50890001322277%
Evaluated Time: 47.39826732476552 minutes


### 4. MLP

In [ ]:
evaluate_trained_model(model_mlp,X_test,y_test_numeric)
print(f"Evaluated Time: {Time_mlp} seconds")

111/111 [==============================] - 0s 2ms/step
Accuracy: 93.35585585585585
Precision: 95.88626739261949
Recall: 90.41642897889332
F1 Score: 93.07105108631826
AUC: 98.08136421102299
Evaluated Time: 10.6338472366333 seconds


### 5. TCN

In [ ]:
evaluate_trained_model(model_tcn,X_test_tcn,y_test_numeric)
print(f"Evaluated Time: {Time_tcn} seconds")

111/111 [==============================] - 0s 4ms/step
Accuracy: 96.19932432432432
Precision: 95.4494382022472
Recall: 96.91956645750143
F1 Score: 96.17888480045288
AUC: 98.7713590011818
Evaluated Time: 50.606406450271606 seconds
